In [9]:
import glob

from elasticsearch.client import IndicesClient, Elasticsearch

INDEX = 'ustawy'
TYPE = 'text'

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

ic = IndicesClient(es)

if ic.exists(index=INDEX):
    ic.delete(INDEX)
ic.create(index=INDEX, body={
    "settings": {
        "analysis": {
            "analyzer": {
                "custom_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "morfologik_stem",
                        "lowercase"
                    ]
                }
            },
            "filter": {
                "synonyms": {
                    "type": "synonym",
                    "synonyms": [
                        "kpk => kodeks postępowania karnego",
                        "kpc => kodeks postępowania cywilnego",
                        "kk => kodeks karny",
                        "kc => kodeks cywilny",
                    ]
                }
            }
        }
    },
    "mappings": {
        "text": {
            "properties": {
                "text": {
                    "type": "text",
                    "analyzer": "custom_analyzer"
                }
            }
        }
    }
})

list_of_files = glob.glob('../ustawy/*.txt')  # create the list of file

print("loading files....")

for file_name in list_of_files:
    with open(file_name, 'r') as myfile:
        data = myfile.read()
        es.index(index=INDEX, doc_type=TYPE, id=file_name, body={
            "text": data,
        })

print("\nNumber of legislative acts containing the word ustawa (in any form):\t{}".format(
    es.search(index=INDEX, doc_type=TYPE, body={
        "query": {
            "match": {
                "text": {
                    "query": "ustawa",
                }
            }
        }
    })['hits']['total']))

print("\nNumber of legislative acts containing the words kodeks postępowania cywilnego in the specified order, "
      "but in an any inflection form:\t{}".format(
    es.search(index=INDEX, doc_type=TYPE, body={
        "query": {
            "match_phrase": {
                "text": {
                    "query": "kodeks postępowania cywilnego",
                }
            }
        }
    })['hits']['total']))

print("\nNumber of legislative acts containing the words wchodzi w życie (in any form) allowing for up to 2 additional "
      "words in the searched phrase:\t{}".format(
    es.search(index=INDEX, doc_type=TYPE, body={
        "query": {
            "match_phrase": {
                "text": {
                    "query": "wchodzi w życie",
                    "slop": 2
                }
            }
        }
    })['hits']['total']))

constitutions = sorted(es.search(index=INDEX, body={
    "query": {
        "match": {
            "text": {
                "query": "konstytucja",
            }
        }
    },
    "highlight": {
        "fields": {
            "text": {}
        },
        "number_of_fragments": 3
    }
})['hits']['hits'], key=lambda x: -x["_score"])[:10]

print("\nmost relevant for the phrase konstytucja: \n")

for i in constitutions:
    print("{}:\n{}\n{}\n".format(i['_id'], i["_score"], i['highlight']['text']))


loading files....

Number of legislative acts containing the word ustawa (in any form):	1157

Number of legislative acts containing the words kodeks postępowania cywilnego in the specified order, but in an any inflection form:	97

Number of legislative acts containing the words wchodzi w życie (in any form) allowing for up to 2 additional words in the searched phrase:	1153

most relevant for the phrase konstytucja: 

../ustawy/2000_443.txt:
7.314219
['umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89\n     ust. 1 lub art. 90 <em>Konstytucji</em>', 'międzynarodowej lub załącznika nie\n     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>', 'co do zasadności wyboru\n  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2\n  <em>Konstytucji</em>']

../ustawy/1997_629.txt:
6.602116
['o zmianie ustawy konstytucyjnej o trybie przygotowania\n           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej', 'W ustawie  konsty